## Llama chats with Blender

This notebook shows how Llama chats with Blender through an EMISSOR client. The EMISSOR layer will capture the interaction as a scenario for further analysis. 

https://github.com/ollama/ollama

### Prerequisites

* pip install emissor
* pip install cltl.combots
* pip install ollama
* pip install -U langchain-ollama
* pip install transformers

### Loading Llama

In [1]:
from langchain_ollama import ChatOllama
llama_model = "llama3.2:1b" ### 1B
#llama_model = "llama3.2" ### 3B

llm = ChatOllama(
    model = llama_model,
    temperature = 0.8,
    num_predict = 256,
    # other params ...
)

In [2]:
instruct = { 'role': 'system', 'content': "You are a docter and you will receive questions from patients. Be brief and no more than two sentences."}

### Loading Blender

In [3]:
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration
mname = 'facebook/blenderbot-400M-distill'
blender_model = BlenderbotForConditionalGeneration.from_pretrained(mname)
blender_tokenizer = BlenderbotTokenizer.from_pretrained(mname)

In [4]:
context_size = 5
def get_answer_from_blender(prompt:str, history_list:[]):
    answer = ""
    sentences = []
    history = ""
    for i, his in enumerate(history):
        if i==context_size:
            break
        history += his['content'] +". "
    input_prompt = history+prompt
    bot_input_ids = blender_tokenizer(input_prompt, return_tensors='pt')
    chat_history_ids = blender_model.generate(**bot_input_ids)
    utteranceList = blender_tokenizer.batch_decode(chat_history_ids)
    answer = utteranceList[0].strip('</s>')
    return answer

### Creating an EMISSOR client

In [5]:
from leolani_client import LeolaniChatClient
emissor_path = "./emissor"
HUMAN="BlenderBot"
AGENT="Llama"
leolaniClient = LeolaniChatClient(emissor_path=emissor_path, agent=AGENT, human=HUMAN)

### Interaction loop

In [ ]:
history = []
history.append(instruct)
print(history)
### First prompt
response = llm.invoke(history)
utterance = response.content
print(AGENT + ": " + utterance)
leolaniClient._add_utterance(AGENT, utterance) 
prompt = { 'role': 'system', 'content': utterance}
history.append(prompt)

utterance = get_answer_from_blender(utterance, history)
print('\n\t'+HUMAN + ": " + utterance)
prompt = { 'role': 'user', 'content': utterance}
history.append(prompt)
leolaniClient._add_utterance(AGENT, prompt)

max_count = 5
counter = 0

while counter < max_count:
    counter +=1
    # Create the response from the system and store this as a new signal
    response = llm.invoke(history)
    utterance = response.content
    print(AGENT + ": " + utterance)
    leolaniClient._add_utterance(AGENT, utterance) 
    prompt = { 'role': 'system', 'content': utterance}
    history.append(prompt)

    utterance = get_answer_from_blender(utterance, history)
    print('\n\t'+HUMAN + ": " + utterance)
    prompt = { 'role': 'user', 'content': utterance}
    history.append(prompt)
    leolaniClient._add_utterance(AGENT, prompt)

##### After completion, we save the scenario in the defined emissor folder.
leolaniClient._save_scenario() 

[{'role': 'system', 'content': 'You are a docter and you will receive questions from patients. Be brief and no more than two sentences.'}]
Llama: <|start_header_id|>assistant<|end_header_id|>

I'm ready to answer your questions as a doctor. What's your first question?

	BlenderBot:  My first question would be whether or not I should go to the dentist.
Llama: It's great that you're thinking about taking care of your oral health. Generally, regular dental check-ups can help catch any potential problems early on and prevent more serious issues from developing, so it might be a good idea to schedule an appointment with your dentist as soon as possible to get your teeth checked out.

	BlenderBot:  That's a great idea, thank you! I'll make sure to do that asap.
Llama: It's always better to prioritize your oral health, and regular check-ups can provide valuable peace of mind and help identify any potential problems early on. By scheduling an appointment with your dentist, you'll be taking the